# Assignment 4 

## Initialization

Import required libraries.

In [13]:
# Standard ML libaries
import random
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report

# RobertaModel and Tockenizer
from transformers import RobertaTokenizer, RobertaModel


Initialize environment with GPU (or CPU as fallback!).

In [14]:
# enable tqdm in pandas
tqdm.pandas()

# set to True to use the gpu (if there is one available)
use_gpu = True

# select device
device = torch.device('cuda' if use_gpu and torch.cuda.is_available() else 'cpu')
print(f'device: {device.type}')

# random seed
seed = 1234

# set random seed
if seed is not None:
    print(f'random seed: {seed}')
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)


device: cpu
random seed: 1234


## Data Processing

Recombine dataset (github quirk)

In [ ]:
import os

try open("dataset.txt", 'r') as file
    file.close()
except e: 
    print(e + " Attempting to reconstruct dataset.")

Load in dataset.

In [15]:
dataset = []

linecount = 0
wordcount = 0

with open("dataset.txt", 'r') as dataset_file:
    while line := dataset_file.readline():
        dataset += [line]
        linecount += 1
        wordcount += len(line)

print("Loaded " + str(linecount) + " lines and " + str(wordcount) + " words.")

Loaded 4468825 lines and 300935040 words.


Initialize tokenizer and then prepare the dataset by A) Generating a tokenization and B) Generating a vocabulary list

In [31]:
tokenizer = RobertaTokenizer.from_pretrained("FacebookAI/roberta-base", add_prefix_space = True, clean_up_tokenization_spaces = True)

In [56]:
print(dataset[0])
token_set = tokenizer(dataset, is_split_into_words=True)["input_ids"]
token_set
# " ".join(tokenizer.convert_ids_to_tokens(token_set)).replace("Ġ", '')

The White Monkey is a 1925 American silent drama film, directed by Phil Rosen and starring Barbara La Marr, Thomas Holding, and Henry Victor.



Token indices sequence length is longer than the specified maximum sequence length for this model (71946167 > 512). Running this sequence through the model will result in indexing errors


[0,
 20,
 735,
 34546,
 16,
 10,
 36248,
 470,
 8454,
 4149,
 822,
 6,
 3660,
 30,
 4720,
 11998,
 8,
 8996,
 4810,
 1587,
 1127,
 338,
 6,
 1813,
 11549,
 6,
 8,
 4858,
 8804,
 4,
 50118,
 85,
 21,
 703,
 30,
 1234,
 496,
 12290,
 15,
 502,
 262,
 6,
 36248,
 4,
 50118,
 39154,
 50118,
 287,
 1602,
 11,
 10,
 822,
 4320,
 1551,
 6,
 12602,
 710,
 6,
 1354,
 9,
 407,
 12336,
 18095,
 219,
 859,
 6,
 4401,
 4458,
 988,
 4436,
 6,
 1060,
 275,
 1441,
 3884,
 506,
 10505,
 15071,
 6,
 2730,
 6,
 25760,
 6,
 8,
 33489,
 6,
 16,
 67,
 11,
 657,
 19,
 69,
 4,
 50118,
 91,
 2215,
 14,
 12602,
 710,
 2997,
 988,
 396,
 143,
 657,
 13,
 123,
 4,
 50118,
 520,
 12602,
 710,
 16766,
 7,
 1157,
 123,
 7,
 989,
 928,
 6,
 37,
 11703,
 14,
 37,
 34,
 10,
 778,
 7,
 339,
 69,
 4,
 50118,
 3884,
 506,
 10505,
 3026,
 988,
 14,
 37,
 40,
 339,
 69,
 409,
 31,
 123,
 114,
 37,
 64,
 4,
 50118,
 988,
 6,
 67,
 9,
 5,
 2706,
 71,
 5,
 997,
 6,
 16766,
 7,
 39705,
 1755,
 12602,
 710,
 50,
 2120,
 143,
 61